# Vector Similarity for RediSearch

This file contains the different capabilities RediSearch offers in the private preview build.
RediSearch vector similiarity capabilities are:
1. Realtime vector indexing
2. Realtime vector update
3. Realtime vector deletion
4. TOP-K query

## Indexing capabilities
In private preview build there are two types of indexing methods supported and three types of distance metrics:

### Index algorithms
1. Brute force (Flat Index)
2. HNSW (Hierarchical Navigable Small World)

### Distance metrics
1. L2 - Euclidean distance between two vectors
2. IP - Internal product of two vectors
3. COSINE - Cosine similarity of two vectors

## Creating an index
In order to create a vector index, the index creation command `FT.CREATE` should be invoked over the vector field name with the new reserved word `VECTOR`

Command format:
```
FT.CREATE <index_name> SCHEMA <vector field name> VECTOR <type> <num params> [<algorithm parameter name> <parameter value> ...]
```
### General indexing mandatory parameters
Parameters should be given to the index build command in the following order
* type - vector data type -  - Currently only `FLOAT32` is supported
* dimension - vector dimension.
* distance metric - either `L2` for euclidean distance, `IP` for internal product or `COSINE` for cosine similarity should be provided. Note, when `COSINE` is selected the indexed vectors will be normalized upon indexing, and the query vector will be normalized upon query.
* Indexing algorithm - either `FLAT` for brute force or `HNSW` for HNSW indexing algorithm

### Brute force (Flat index)
This index compares the entire indexed vector data to the query vector and returns the top-k similar vectors, according to the given distance metric.

* **Mandatory parameters**

    * **TYPE** - 
        Vector type. Current supported type is `FLOAT32`.
    
    * **DIM** - 
        Vector dimention. should be positive integer.
    
    * **DISTANCE_METRIC** - 
        Supported distance metric. Currently one of **{`L2`, `IP`, `COSINE`}**

* **Optional parameters**

    * **INITIAL_CAP** - 
        Initial vector capacity in the index. Effects memory allocation size of the index.

    * **BLOCK_SIZE** - 
        block size to hold `BLOCK_SIZE` amount of vectors in a contiguous array.
        This is useful when the index is dynamic with respect to addition and deletion.
        Defaults to 1048576 (1024*1024).

* Example

    ```
    FT.CREATE my_index1 
    SCHEMA vector_field VECTOR 
    FLAT 
    10 
    TYPE FLOAT32 
    DIM 128 
    DISTANCE_METRIC L2 
    INITIAL_CAP 1000000 
    BLOCK_SIZE 1000
    ```

### HNSW
This index algorithm is a modified version of [nmslib/hnswlib](https://github.com/nmslib/hnswlib) which is the author's implementation of [Efficient and robust approximate nearest neighbor search using Hierarchical Navigable Small World graphs](https://arxiv.org/ftp/arxiv/papers/1603/1603.09320.pdf)

* **Mandatory parameters**

    * **TYPE** - 
        Vector type. Current supported type is `FLOAT32`.
    
    * **DIM** - 
        Vector dimention. should be positive integer.
    
    * **DISTANCE_METRIC** - 
        Supported distance metric. Currently one of **{`L2`, `IP`, `COSINE`}**

* **Optional parameters**

    * **INITIAL_CAP** - 
        Initial vector capacity in the index. Effects memory allocation size of the index.

    * **M** - 
        Number the maximal allowed outgoing edges for each node in the graph. Defaults to 16.

    * **EF_CONSTRUCTION** - 
        Number the maximal allowed potential outgoing edges candidates for each node in the graph, during the graph building. Defaults to 200.

    * **EF_RUNTIME** - 
        Number the maximal allowed potential candidates during the top-K query. Defaults to 10.

* Example

    ```
    FT.CREATE my_index2 
    SCHEMA vector_field VECTOR 
    HNSW 
    14 
    TYPE FLOAT32 
    DIM 128 
    DISTANCE_METRIC L2 
    INITIAL_CAP 1000000 
    M 40 
    EF_CONSTRUCTION 250 
    EF_RUNTIME 20
    ```

## Query
In order to execute top-k vector query the search command `FT.SEARCH` should be invoked with the vector blob as a parameter

Command format:
```
FT.SEARCH <index name> "*=>[TOP_K <k> @my_vector_field $<vector blob parameter name> ]" SORTBY <vector field name>_score PARAMS 2 <vector blob parameter name> <vector blob>
```
```
FT.SEARCH <index name> "*=>[TOP_K <k> @my_vector_field $<vector blob parameter name> AS <score field name> ]" SORTBY <score field name> PARAMS 2 <vector blob parameter name> <vector blob>
```

### Query tuning parameters
#### HNSW
* `EF_RUNTIME` - Maximum number of potential top-k candidates to collect while querying the graph. `EF_RUNTIME` should be greater or equal to `K`

An example for top-10 query over HNSW indexed dataset with `EF_RUNTIME` equals 150

```
FT.SEARCH my_hnsw_index "*=>[TOP_K 10 @my_vector_field $vec EF_RUNTIME 150]" SORTBY my_vector_field_score LIMIT 0 10 PARAMS 2 vec <vector blob>
```

## Python examples

### Packages

In [9]:
!pip install git+https://github.com/RediSearch/redisearch-py.git@params
!pip install numpy

  Cloning https://github.com/RediSearch/redisearch-py.git (to revision params) to /tmp/pip-req-build-a3f7k_u3
  Running command git clone -q https://github.com/RediSearch/redisearch-py.git /tmp/pip-req-build-a3f7k_u3
  Running command git checkout -b params --track origin/params
  Switched to a new branch 'params'
  Branch 'params' set up to track remote branch 'params' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for redisearch: filename=redisearch-2.0.0-py2.py3-none-any.whl size=26773 sha256=df265a22a9f26fd63895ad2898e0a5bb8776a74ce68b7ea5841e53f00e475e9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-7pn8k1g1/wheels/b4/7c/ba/01a2c3649eba62bf60cf6e2c61f86b18922cb53b17b1c8366b
  ERROR: Command errored out with exit status 1:
   command: /home/dvirdukhan/Code/RediSearch/venv/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-8vvi4o31/re

In [10]:
import numpy as np
from redis import Redis
import redisearch

Create redis client

In [11]:
host = "localhost"
port = 6379

redis_conn = Redis(host = host, port = port)

In [12]:
n_vec = 10000
dim = 128
M = 40
EF = 200
vector_field_name = "vector"
k = 10
hnsw_EFRUNTIME = 10

In [13]:
def load_vectors(client : Redis, n, d,  field_name):
    for i in range(n):
        np_vector = np.random.rand(1, d).astype(np.float32)
        client.hset(i, field_name, np_vector.tobytes())
        
def delete_data(client: Redis):
    client.flushall()
        

### Brute Force

In [14]:
# build index
bf_index = redisearch.Client("my_flat_index", conn=redis_conn)
bf_index.redis.execute_command("FT.CREATE", "my_flat_index", "SCHEMA", vector_field_name, "VECTOR", "FLAT", "8", "TYPE", "FLOAT32", "DIM", dim, "DISTANCE_METRIC", "L2", "INITIAL_CAP", n_vec)
#load vectors
load_vectors(bf_index.redis, n_vec, dim, vector_field_name)
#query
query_vector =  np.random.rand(1, dim).astype(np.float32)
q = redisearch.Query(f'*=>[TOP_K {k} @{vector_field_name} $vec_param]').sort_by(f'{vector_field_name}_score').paging(0,k).return_field(f'{vector_field_name}_score')
res = bf_index.search(q, query_params = {'vec_param': query_vector.tobytes()})
docs = [int(doc.id) for doc in res.docs]
rs_dists = [float(doc.vector_score) for doc in res.docs]
print(docs)
print(rs_dists)
#cleanup
delete_data(bf_index.redis)

[7292, 5031, 1744, 2473, 331, 6175, 6966, 8245, 9556, 5081]
[14.4533643723, 14.7699546814, 14.7886180878, 14.8911895752, 15.1060228348, 15.2433671951, 15.4294042587, 15.4431753159, 15.4483947754, 15.5052080154]


### HNSW

In [15]:
# build index
hnsw_index = redisearch.Client("my_hnsw_index", conn=redis_conn)
hnsw_index.redis.execute_command("FT.CREATE", "my_hnsw_index", "SCHEMA", vector_field_name, "VECTOR", "HNSW", "12", "TYPE", "FLOAT32", "DIM", dim, "DISTANCE_METRIC", "L2", "INITIAL_CAP", n_vec, "M", M, "EF_CONSTRUCTION", EF)
#load vectors
load_vectors(hnsw_index.redis, n_vec, dim, vector_field_name)
#query
query_vector =  np.random.rand(1, dim).astype(np.float32)
q = redisearch.Query(f'*=>[TOP_K {k} @{vector_field_name} $vec_param EF_RUNTIME {hnsw_EFRUNTIME}]').sort_by(f'{vector_field_name}_score').paging(0,k).return_field(f'{vector_field_name}_score')
res = hnsw_index.search(q, query_params = {'vec_param': query_vector.tobytes()})
docs = [int(doc.id) for doc in res.docs]
rs_dists = [float(doc.vector_score) for doc in res.docs]
print(docs)
print(rs_dists)
#cleanup
delete_data(hnsw_index.redis)

[5084, 4905, 2596, 9570, 4727, 9640, 4627, 7197, 2826, 7182]
[14.4682731628, 15.0336112976, 15.1415758133, 15.2895421982, 15.3157777786, 15.3399477005, 15.478852272, 15.6448297501, 15.6617927551, 15.9710283279]
